In [1]:
%matplotlib inline
import tensorflow as tf
import matplotlib as plt
import glob
import numpy as np
from PIL import Image
import mltools as ml

trying to use the example from 
[http://warmspringwinds.github.io/tensorflow/tf-slim/2016/12/21/tfrecords-guide/](http://warmspringwinds.github.io/tensorflow/tf-slim/2016/12/21/tfrecords-guide/)

**biome dataset**:  
denote number of data  
mesa size:  10015  
forest size:  10047  
desert size:  10106  
jungle size:  10238  
eh size:  7364  
training size: 42993  
test size: 4777  
total size:  47770

**pig dataset:**  
denote number of data  
fence: 433  
pig: 404 
training size: 669  
test size: 168  
total size:  837

In [2]:
# labels ={"mesa":0, "forest":1, "desert":2,"jungle":3,"eh":4}
labels = {"fence":0, "pig":1}
tfrecords_filename = '/notebooks/Minecraft-AI/mc-data/fence_pig_train.tfrecords'
tfrecords_test_filename='/notebooks/Minecraft-AI/mc-data/fence_pig_test.tfrecords'

In [3]:
files_paths = [
    ('/notebooks/Minecraft-AI/mc-data/fence_rgb_2/*.jpg', "0"),
    ('/notebooks/Minecraft-AI/mc-data/pig_rgb_2/*.jpg', "1")
]

In [4]:
filename_pairs=[]
for file_name, label in files_paths:
    files = glob.glob(file_name)
    file_pairs = zip(files, label*len(files))
    print "label: ", label, " size: ", len(files)
    filename_pairs += file_pairs

label:  0  size:  433
label:  1  size:  404


In [5]:
np.random.shuffle(filename_pairs)
portion=int(0.8*len(filename_pairs))
train_pairs = filename_pairs[:portion]
test_pairs = filename_pairs[portion:]

In [6]:
print train_pairs[:10]

[('/notebooks/Minecraft-AI/mc-data/pig_rgb_2/pig_132.jpg', '1'), ('/notebooks/Minecraft-AI/mc-data/fence_rgb_2/fence_674.jpg', '0'), ('/notebooks/Minecraft-AI/mc-data/fence_rgb_2/fence_510.jpg', '0'), ('/notebooks/Minecraft-AI/mc-data/pig_rgb_2/pig_425.jpg', '1'), ('/notebooks/Minecraft-AI/mc-data/pig_rgb_2/pig_283.jpg', '1'), ('/notebooks/Minecraft-AI/mc-data/fence_rgb_2/fence_258.jpg', '0'), ('/notebooks/Minecraft-AI/mc-data/fence_rgb_2/fence_504.jpg', '0'), ('/notebooks/Minecraft-AI/mc-data/fence_rgb_2/fence_465.jpg', '0'), ('/notebooks/Minecraft-AI/mc-data/fence_rgb_2/fence_678.jpg', '0'), ('/notebooks/Minecraft-AI/mc-data/pig_rgb_2/pig_23.jpg', '1')]


In [7]:
print "training size:", len(train_pairs)
print "test size:", len(test_pairs)
print "total size: ", len(filename_pairs)

training size: 669
test size: 168
total size:  837


In [8]:

train_writer = tf.python_io.TFRecordWriter(tfrecords_filename)
test_writer = tf.python_io.TFRecordWriter(tfrecords_test_filename)

In [9]:
# helper functions
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [10]:
def write_in_tfrecord(filename_pairs, writer):
    for img_path, label in filename_pairs:

        img = np.array(Image.open(img_path))
        
        # The reason to store image sizes was demonstrated
        # in the previous example -- we have to know sizes
        # of images to later read raw serialized string,
        # convert to 1d array and convert to respective
        # shape that image used to have.
        height = img.shape[0]
        width = img.shape[1]

        # Put in the original images into array
        # Just for future check for correctness
#         original_images.append((img,label))

        img_raw = img.tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(height),
            'width': _int64_feature(width),
            'image_raw': _bytes_feature(img_raw),
            'label': _int64_feature(int(label))}))

        writer.write(example.SerializeToString())

In [11]:
write_in_tfrecord(train_pairs,train_writer)
train_writer.close()

In [12]:
write_in_tfrecord(test_pairs,test_writer)
test_writer.close()